In [0]:
import os
import json
import requests
import delta
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, ArrayType

import sys
sys.path.append('../..')
from ETL.Common.api import call_data_gov_api

In [0]:
%run ../../Utils/spark_init

In [0]:
dir = os.getcwd()
uri = "/resource/81153f15-b4da-45b5-a299-f307351c5001"
data_gov_url = "https://data.gov.in/resource/list-lok-sabha-members-english"
data_gov_name = "List of Lok Sabha Members (English)"
table_name = "ls_members"

resp = call_data_gov_api(uri)
# print(resp.json()['records'])

json_array_schema = StructType([
    StructField('sno', StringType(), nullable=False), 
    StructField('memberid', StringType(), nullable=False),
    StructField('name', StringType(), nullable=False),
    StructField('party', StringType(), nullable=False),
    StructField('constituencies', StringType(), nullable=False),
    StructField('state', StringType(), nullable=False), 
    StructField('paddress', StringType(), nullable=False),
    StructField('permanentphone', StringType(), nullable=False),
    StructField('localaddress', StringType(), nullable=False),
    StructField('localphone', StringType(), nullable=False),
    StructField('emailid', StringType(), nullable=False)
  ])

# Read as a dictionary
df = spark.createDataFrame(data=resp.json()['records'], schema = json_array_schema)

# # The following  only stores data in delta file format
# df.write.format("delta").mode("overwrite").save(dir + os.sep + "/data/ls_members")
# df.show(truncate=True)

df.write.mode("overwrite").saveAsTable(table_name)

display(spark.sql('DESCRIBE DETAIL default.{};'.format(table_name)))
display(spark.sql("SELECT * FROM default.{};".format(table_name)))